In [38]:
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os
import yaml


def read_file(file_path):
    with open(file_path) as yaml_file:
        file=yaml.safe_load(yaml_file)
    return file

def log_production_model(config_path):
    config = read_file(config_path)
    
    
    mlflow_config = config["mlflow_config"] 
    

    model_name = mlflow_config["registered_model_name"]


    remote_server_uri = mlflow_config["remote_server_uri"]
    
    mlflow.set_tracking_uri(remote_server_uri)
    
    runs = mlflow.search_runs(experiment_ids=1)
        
    ####### mention you metrics name checking its name in mlflow dashboard#########
    
    lowest = runs["metrics.training_mae"].sort_values(ascending=True)[0]
    print(runs)
    
    lowest_run_id = runs[runs["metrics.training_mae"] == lowest]["run_id"][0]
    
    client = MlflowClient()
    
    
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)
        
        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            print(logged_model)
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="Production"
            )
        else:
            current_version = mv["version"]
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="Staging"
            )        

log_production_model("/home/shubham/car_dekho_project/Car_Dekho/params.yaml")

                              run_id experiment_id    status  \
0   290660aa3fa4469e9a3c9091bd3bc9c7             1  FINISHED   
1   63b1be0c45934cea954934829bb06271             1  FINISHED   
2   71043315d5604129a9d5705e4882eb25             1  FINISHED   
3   9fa7dfdad4e74a09b3fc59119684aeab             1  FINISHED   
4   b4541c644cd44b309e462bf13ba4b509             1  FINISHED   
5   f3869b39b3af43e5a97b5b3bf5a2adf1             1  FINISHED   
6   2197ef5fa14b450f97c1cca3dc93a13c             1  FINISHED   
7   4ed1bb8cf5514bbeb14305de4d1f132c             1  FINISHED   
8   569ebf12aeba4500a332e4dea0496b62             1  FINISHED   
9   64b790393f7f464a8442cf9bf0e6e83c             1  FINISHED   
10  6f4ad766a1304f5493994e06ffa8fc21             1  FINISHED   
11  d46c378ac8b44d05813691cb599a870c             1  FINISHED   

                                         artifact_uri  \
0   ./artifacts/1/290660aa3fa4469e9a3c9091bd3bc9c7...   
1   ./artifacts/1/63b1be0c45934cea954934829bb06271...

KeyError: 0

In [28]:

loaded_model = mlflow.pyfunc.load_model(logged_model)

model_path = config["webapp_model_dir"] #"prediction_service/model"

joblib.dump(loaded_model, model_path)

NameError: name 'logged_model' is not defined

In [34]:
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os
import yaml


def read_file(file_path):
    with open(file_path) as yaml_file:
        file=yaml.safe_load(yaml_file)
    return file

def log_production_model(config_path):
    config = read_file(config_path)
    
    
    mlflow_config = config["mlflow_config"] 
    

    model_name = mlflow_config["registered_model_name"]


    remote_server_uri = mlflow_config["remote_server_uri"]
    
    mlflow.set_tracking_uri(remote_server_uri)
    
    runs = mlflow.search_runs(experiment_ids=1)
        
    ####### mention you metrics name checking its name in mlflow dashboard#########
    
    lowest = runs["metrics.training_mae"].sort_values(ascending=True)[0]
    
    lowest_run_id = runs[runs["metrics.training_mae"] == lowest]["run_id"][0]
    
    client = MlflowClient()    
    
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)
        pprint(mv)
        # if mv["run_id"] == lowest_run_id:
        #     current_version = mv["version"]
        #     logged_model = mv["source"]
        #     print(logged_model)
        #     pprint(mv, indent=4)
        #     client.transition_model_version_stage(
        #         name=model_name,
        #         version=current_version,
        #         stage="Production"
        #     )
        # else:
        #     current_version = mv["version"]
        #     client.transition_model_version_stage(
        #         name=model_name,
        #         version=current_version,
        #         stage="Staging"
        #     )        

log_production_model("/home/shubham/car_dekho_project/Car_Dekho/params.yaml")

KeyError: 0